# Quantum Support Vector Machine Cloud Detection
1. Data Preprocessing

In [1]:
import numpy as np
# Grab Dataset
data = np.genfromtxt('../data.csv', delimiter=',')

In [2]:
# Split data into y (label) and x (features)
y = data[:,0]
x = data[:,1:22]
x = np.nan_to_num(x, nan=0.0)

In [3]:
# Any multi-layer cloud labels set to 1, turn into classification [cloud, no cloud]=[1,0]
y = np.where(y >= 1, 1, y)
print(y[0])

0.0


In [4]:
# Generate 10000 random indices
np.random.seed(7)
random_indices = np.random.choice(len(x), size=10000, replace=False)

# Select samples using the random indices
x = x[random_indices,:]
y = y[random_indices]

In [5]:
# cut down on features
# 0 and 6 were chosen from Random Forest's Feature Importance
x = x[:,[0,6]]

In [6]:
from sklearn.model_selection import train_test_split
# Split data into Train/Test/Validation 10/90
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.9, random_state=42)

# Reduce test set to cut down processing cost, running test score is costly
X_test = X_test[750:1000,:]
y_test = y_test[750:1000]

In [7]:
from sklearn.preprocessing import StandardScaler
# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
print(x.shape[1])
print(np.shape(X_train))
print(np.max(X_train[:,0]))
print(np.min(X_train[:,1]))
print(np.max(X_train[:,0]))
print(np.min(X_train[:,1]))

2
(1000, 2)
6.8964046359257605
-2.279219008595724
6.8964046359257605
-2.279219008595724


2. Build Quantum SVM

In [9]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

adhoc_feature_map = ZZFeatureMap(feature_dimension=x.shape[1], reps=3, entanglement="linear")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)

3. Train

In [10]:
from qiskit_machine_learning.algorithms import QSVC
import time

qsvc = QSVC(quantum_kernel=adhoc_kernel)

start_fit = time.time()
qsvc.fit(X_train, y_train)
end_fit = time.time()
fit_time = end_fit - start_fit

print(f"Time elapsed for fitting: {fit_time} seconds")

Time elapsed for fitting: 2515.318550348282 seconds


In [11]:
start_score = time.time()
qsvc_score = qsvc.score(X_test, y_test)
end_score = time.time()
score_time = end_score - start_score

print(f"Time elapsed for scoring: {score_time} seconds")
print(f"QSVC classification test score: {qsvc_score}")

Time elapsed for scoring: 1274.2756481170654 seconds
QSVC classification test score: 0.748
